# Pipeline to run all files
* All scripts currently output Excel files because we were working independently
* <b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [11]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date (leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(0,"","",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder


Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-02-12T06:50:00 to 2019-02-13T06:50:00
160


In [12]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [13]:
articleDB

,title,description,url,source,date,content,origContent
0,The best foundation brushes you can buy,The Insider Picks team writes about stuff we t...,https://www.businessinsider.com/best-foundatio...,business-insider,2019-02-12T15:43:00Z,Why youll love it The Sigma Beauty Flat Kabuki...,Why you'll love it: The Sigma Beauty Flat Kabu...
1,New York Needs Amazon,History is clear: A city that rejects economic...,https://www.nytimes.com/2019/02/12/opinion/ama...,the-new-york-times,2019-02-12T17:15:44Z,The potential loss of the Amazon headquarters ...,The potential loss of the Amazon headquarters ...
2,The Fitbit Versa smartwatch has all the fitnes...,The Insider Picks team writes about stuff we t...,https://www.businessinsider.com/fitbit-versa-s...,business-insider,2019-02-12T21:30:00Z,The Insider Picks team writes about stuff we t...,The Insider Picks team writes about stuff we t...
3,CVS introduces new concept store with more hea...,CVS Health is testing a new concept store form...,https://www.usatoday.com/story/money/2019/02/1...,usa-today,2019-02-13T05:01:00Z,CVS plans to offer a new nationwide telemedici...,CVS plans to offer a new nationwide telemedici...
4,Save up to 25% on last-minute Valentine's Day ...,The Insider Picks team writes about stuff we t...,https://www.businessinsider.com/deal-of-the-da...,business-insider,2019-02-12T17:23:00Z,The Insider Picks team writes about stuff we t...,The Insider Picks team writes about stuff we t...
5,21 helpful products we've bought for our apart...,The Insider Picks team writes about stuff we t...,https://www.businessinsider.com/helpful-apartm...,business-insider,2019-02-12T17:07:00Z,The Insider Picks team writes about stuff we t...,The Insider Picks team writes about stuff we t...
6,Amazon is launching a live-streaming video ver...,This story was delivered to Business Insider I...,https://www.businessinsider.com/amazon-launche...,business-insider,2019-02-12T15:42:00Z,This story was delivered to Business Insider I...,This story was delivered to Business Insider I...
7,"Celebrate Valentine's Day with free champagne,...",Save on your Valentine's Day meals with these ...,https://www.usatoday.com/story/money/2019/02/1...,usa-today,2019-02-12T13:36:17Z,Toast your sweetheart with a glass of free cha...,Toast your sweetheart with a glass of free cha...
8,Every product in this skin-care line by MIT gr...,The Insider Picks team writes about stuff we t...,https://www.businessinsider.com/maelove-skin-c...,business-insider,2019-02-12T21:11:00Z,The Insider Picks team writes about stuff we t...,The Insider Picks team writes about stuff we t...
9,Brian Hoyer's Amazon and ESPN+ subscriptions h...,Brian Hoyer's tips on Sean McVay's offense hel...,https://ftw.usatoday.com/2019/02/super-bowl-53...,usa-today,2019-02-12T12:16:51Z,The biggest play of Super Bowl LIII was made p...,The biggest play of Super Bowl LIII was made p...


In [14]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
binaryMatrix = fe.encoding(0, df=articleDB)

100%|██████████| 149/149 [00:00<00:00, 3563.09it/s]

Binary Encoding


In [15]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(binaryMatrix, articleDB)

                                               title  \
0  Excerpts from recent editorials in newspapers ...   
1  30 acres once owned by Abraham Lincoln in Illi...   
2  Cushy hotel rooms and fussy amenities in luxur...   
3     Can the middle-class revival under Trump last?   
4  Billionaire investor Ray Dalio sat down with h...   

                                         description  
0                                  February 10, 2019  
1  30 acres once owned by Abraham Lincoln in Illi...  
2  Some high-end hotels are making a move towards...  
3  In his State of the Union, Trump said the midd...  
4  In an exclusive interview conducted by Bridgew...  


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [16]:
# 1) Extract key phrases, unigrams, and uni+bigrams
# complete final file is called 'results_context.xlsx'
# first input is filename 
# (optional) second input is tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
#    future work: add in noun phrases, named entities
# 2) outputs "trending_terms.xlsx" which outputs keyterms by # article mentions
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|██████████| 149/149 [00:11<00:00, 12.94it/s]


In [6]:
import Recommender as rec
FinalTable = rec.recommender("binEncoding.csv", "results_context.xlsx")

0
[ 88  32 102   0]
[ 88  32 102   0]
[0.38177086 0.41602515 0.49613894 1.        ]
1
[ 49 132  76   1]
[ 49 132  76   1]
[0.28867513 0.35355339 0.37796447 1.        ]
2
[133  15   2  96]
[133  15  96   2]
[0.3928371  0.42163702 0.4330127  1.        ]
3
[ 49  47  46 143]
[ 49  47  46 143]
[0. 0. 0. 0.]
4
[113   4  84 102]
[113  84 102   4]
[0.33333333 0.4472136  0.51639778 1.        ]
5
[ 69 110  96   5]
[ 69 110  96   5]
[0.38490018 0.40824829 0.41247896 1.        ]
6
[ 99 129  27   6]
[ 99  27 129   6]
[0.35355339 0.35355339 0.42640143 1.        ]
7
[24 23 41  7]
[24 23 41  7]
[0.4472136  0.4472136  0.57735027 1.        ]
8
[13 35 25  8]
[13 25 35  8]
[0.23904572 0.23904572 0.26726124 1.        ]
9
[  7  24 137   9]
[  7  24 137   9]
[0.31139958 0.34815531 0.34815531 1.        ]
10
[28 26 43 10]
[28 26 43 10]
[0.47140452 0.47140452 0.50395263 1.        ]
11
[135  74 102  11]
[135  74 102  11]
[0.36514837 0.39036003 0.4        1.        ]
12
[109 115  76  12]
[109  76 115  12]
[0.3779

In [7]:
FinalTable.head()

,nonRel,Rel,article_id,prediction,difference,title,description,url,source,date,content,origContent,tags,tags_top_5,related_articles
0,-1.696713,-0.202465,93,1,1.494248,Omar smashed over sneers at Jewish money in po...,Omar smashed over sneers at Jewish money in po...,https://www.foxnews.com/politics/omar-smashed-...,fox-news,2019-02-11 23:26:56,Want FOX News Halftime Report in your inbox ev...,**Want FOX News Halftime Report in your inbox ...,"want fox, halftime, congress, democratic leade...","hope,personal biography,horizontal reading,due...","146, 153, 73"
1,-1.365472,-0.294720,150,1,1.070751,Excerpts from recent editorials in newspapers ...,"February 10, 2019",https://apnews.com/1a0371c3dbeb437284d57c3c982...,associated-press,2019-02-12 18:09:15,February 10 2019Chicago TribuneMeasles surges ...,"February 10, 2019\r\nChicago Tribune\r\nMeasle...","tribunemeasles, even death, public demand, her...","forms remuneration,states fatalities,initial r...","146, 134, 128"
2,-1.215559,-0.351750,106,1,0.863809,Cramer: This earnings period revealed 'brutal ...,Jim Cramer guides investors on how to approach...,https://www.cnbc.com/2019/02/11/cramer-earning...,cnbc,2019-02-11 23:11:00,If the latest round of earnings reports taught...,If the latest round of earnings reports taught...,"traders arent, uschina, nike, starbucks, ameri...","pain techs,peoples,unassailable brands,applequ...","44, 117, 103"
3,-0.796482,-0.599497,9,1,0.196985,How to properly clean an iPad screen without d...,You'll want to avoid using sprays or any semi-...,https://www.businessinsider.com/how-to-clean-i...,business-insider,2019-02-11 21:21:00,My device screens are perpetually collecting g...,My device screens are perpetually collecting g...,"device screens, maybe smog, screen ipad, stron...","perimeter ipad,cracks ipad,abrasive cloths,dir...","29, 60, 32"
4,-0.761865,-0.628849,134,1,0.133016,Can the middle-class revival under Trump last?,"In his State of the Union, Trump said the midd...",https://www.usatoday.com/story/money/2019/02/1...,usa-today,2019-02-12 11:01:05,In his first State of the Union address before...,In his first State of the Union address before...,"congress, washington, good country, clear path...","reeves,middle class,highwage jobs,mckinsey,mar...","20, 92, 140"
